## Step 1: imports
Setup all imports needed for your model

In [76]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

## Step 2: Hyper parameters
Setup the hyper parameters for the model

In [77]:
lr = 0.001
epochs = 2
batch_size = 100
input_size = 28 * 28
output_size = 10

## Step 3: Data
Get training and test data ready

In [78]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,)), ])

# training data
mnist_train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(mnist_train_set, batch_size=batch_size, shuffle=True)

# test data
mnist_test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(mnist_test_set, batch_size=batch_size, shuffle=True)

## Step 4: The Model
Design your model and its layers

In [79]:
class GusAI(nn.Module):

    def __init__(self, input_dim=None, output_dim=None):
        super(GusAI, self).__init__()
        self.lin = nn.Linear(input_dim, output_dim)

    def forward(self, image):
        x = torch.flatten(image, 1)
        x = self.lin(x)
        x = f.log_softmax(x, dim=1)
        return x

Maybe use a CNN instead

In [80]:
class NN(nn.Module):

    def __init__(self):
        super(NN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = f.relu(f.max_pool2d(self.conv1(x), 2))
        x = f.relu(f.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = f.relu(self.fc1(x))
        x = f.dropout(x, training=self.training)
        x = self.fc2(x)
        return f.log_softmax(x, dim=1)

In [81]:
class NN2(nn.Module):

    def __init__(self):
        super(NN2, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = f.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Initialise the model, its loss function and optimizer

In [82]:
# model = GusAI(input_dim=input_size, output_dim=output_size)
model = NN2()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

## Step 5: Training Loop
Forward pass, loss, zero grads, backward pass, optimizer step

In [83]:
def train():
    print("========\nTRAINING\n========")
    model.train()
    for epoch in range(epochs):
        for batch, (images, labels) in enumerate(train_loader):
            # forward pass
            output = model(images)

            # loss
            loss = loss_fn(output, labels)

            # zero gradients
            optimizer.zero_grad()

            # gradient
            loss.backward()

            # update weights
            optimizer.step()

        print(f'Epochs [{epoch + 1}/{epochs}], Losses: {loss.item():.4f}')

## Step 6: Testing The Model
Compare prediction to actual labels

In [84]:
 def test():
    print("=======\nTESTING\n=======")
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            output = model(images)
            correct += torch.sum(torch.argmax(output, dim=1) == labels)
            total += labels.shape[0]
        accuracy = correct / total * 100
        print(f'Accuracy of the model  {accuracy:.2f}%')

## Step 7: Run
Run the model and see how it performs

In [85]:
train()
test()

TRAINING
Epochs [1/2], Losses: 0.2784
Epochs [2/2], Losses: 0.2049
TESTING
Accuracy of the model  95.53%
